### YouTube Statistics MySQL Project

#### Importing Necessary Modules

In [1]:
#Importing Necessary Modules
import pandas as pd
import re
import pymysql
from sqlalchemy import create_engine,text
from urllib.parse import quote

#### Importing Datafiles as Pandas Dataframe

In [2]:
# Loading the video-stats file
df_vid = pd.read_csv('videos-stats.csv')

# Loading the comments file
df_comment = pd.read_csv('comments.csv')

In [3]:
df_vid

,Unnamed: 0,Title,Video_ID,Published_At,Keyword,Likes,Comments,Views
0,0,Apple Pay Is Killing the Physical Wallet After...,wAZZ-UWGVHI,23-08-22,tech,3407.0,672.0,135612.0
1,1,The most EXPENSIVE thing I own.,b3x28s61q3c,24-08-22,tech,76779.0,4306.0,1758063.0
2,2,My New House Gaming Setup is SICK!,4mgePWWCAmA,23-08-22,tech,63825.0,3338.0,1564007.0
3,3,Petrol Vs Liquid Nitrogen | Freezing Experimen...,kXiYSI7H2b0,23-08-22,tech,71566.0,1426.0,922918.0
4,4,Best Back to School Tech 2022!,ErMwWXQxHp0,08-08-22,tech,96513.0,5155.0,1855644.0
...,...,...,...,...,...,...,...,...
1876,1876,Should You Learn Machine Learning?,AO6urf07KjE,14-06-21,machine learning,10259.0,416.0,386360.0
1877,1877,Todos podemos aprender Machine learning,7ClLKBUvmRk,08-10-17,machine learning,2981.0,72.0,431421.0
1878,1878,"Andrew Ng: Deep Learning, Education, and Real-...",0jspaMLxBig,20-02-20,machine learning,5198.0,443.0,226152.0
1879,1879,What is Machine Learning?,f_uwKZIAeM0,11-01-17,machine learning,NaN,NaN,NaN


In [4]:
df_comment

,Unnamed: 0,Video_ID,Comment,Likes,Sentiment
0,0,wAZZ-UWGVHI,Let's not forget that Apple Pay in 2014 requir...,95.0,1.0
1,1,wAZZ-UWGVHI,Here in NZ 50% of retailers don’t even have co...,19.0,0.0
2,2,wAZZ-UWGVHI,I will forever acknowledge this channel with t...,161.0,2.0
3,3,wAZZ-UWGVHI,Whenever I go to a place that doesn’t take App...,8.0,0.0
4,4,wAZZ-UWGVHI,"Apple Pay is so convenient, secure, and easy t...",34.0,2.0
...,...,...,...,...,...
18404,18404,cyLWtMSry58,I really like the point about engineering tool...,0.0,2.0
18405,18405,cyLWtMSry58,I’ve just started exploring this field. And th...,20.0,2.0
18406,18406,cyLWtMSry58,Excelente video con una pregunta filosófica pr...,1.0,1.0
18407,18407,cyLWtMSry58,"Hey Daniel, just discovered your channel a cou...",35.0,2.0


#### Database Connection: Connecting Jupyter Notebook to MySQL Server

In [5]:
db_connection = create_engine('mysql+pymysql://root:%s@localhost:3306/youtubestat' % quote('Abcd@1234'))

#### Creating videostat table in MySQL from Pandas Dataframe

In [6]:
df_vid.to_sql(name='videostat', con=db_connection, if_exists='replace', index=False)

1881

#### Creating comment table in MySQL from Pandas Dataframe

In [7]:
df_comment.to_sql(name='comment', con=db_connection, if_exists='replace', index=False)

18409

In [8]:
conn = db_connection

## Queries Mentioned in the Dataquest Project https://www.dataquest.io/blog/sql-projects/

### What are the most commented-upon videos?

#### Query: Select Title,Comments from videostat order by Comments desc

In [9]:
comment_query = text("Select Title,Comments from videostat order by Comments desc")
op = pd.DataFrame(conn.execute(comment_query))
op

,Title,Comments
0,El Chombo - Dame Tu Cosita feat. Cutty Ranks (...,732818.0
1,I Opened A Restaurant That Pays You To Eat At It,728774.0
2,"$456,000 Squid Game In Real Life!",611757.0
3,One Direction - History (Official Video),434688.0
4,"history of the entire world, i guess",395657.0
...,...,...
1876,TOP SECRET MISSION â­ðŸŒŸ Best Action Movies ...,0.0
1877,SURVIVOR ðŸ¦€ðŸ¦‚ Best Action Movies 2022 ðŸ¦€...,-1.0
1878,BEST Auditions Of Songs From Movies | Amazing ...,-1.0
1879,What is Machine Learning?,NaN


### What are the most Liked videos?

#### Query: Select Title,Likes from videostat order by Likes desc

In [10]:
like_query = text("Select Title,Likes from videostat order by Likes desc")
op = pd.DataFrame(conn.execute(like_query))
op

,Title,Likes
0,El Chombo - Dame Tu Cosita feat. Cutty Ranks (...,16445558.0
1,"$456,000 Squid Game In Real Life!",14259033.0
2,Martin Garrix - Animals (Official Video),11025176.0
3,Powfu - death bed (coffee for your head) (Offi...,7786057.0
4,The Weeknd - Save Your Tears (Official Music V...,6823113.0
...,...,...
1876,FULL Marvel Studios Panel from Hall H | San Di...,-1.0
1877,The Magnificent Seven - Full Movie In English ...,-1.0
1878,Vikram Vedha Movie Teaser Review | KRK | #krkr...,-1.0
1879,What is Machine Learning?,NaN


### How many total views does each category have?

#### Query: Select Keyword,sum(Views) as Total_Views from videostat group by Keyword

In [11]:
view_query = text("Select Keyword,sum(Views) as Total_Views from videostat group by Keyword")
op = pd.DataFrame(conn.execute(view_query))
op

,Keyword,Total_Views
0,tech,9.210273e+07
1,news,9.652195e+06
2,gaming,2.234476e+07
3,sports,4.214590e+08
4,how-to,3.748457e+08
5,business,3.473450e+08
6,tutorial,3.380516e+08
7,finance,2.707471e+07
8,crypto,2.023041e+07
9,cubes,7.519370e+08


### How many total like does each category have?

#### Query: Select Keyword,sum(Likes) as Total_Likes from videostat group by Keyword

In [12]:
view_likes = text("Select Keyword,sum(Likes) as Total_Likes from videostat group by Keyword")
op = pd.DataFrame(conn.execute(view_likes))
op

,Keyword,Total_Likes
0,tech,2905652.0
1,news,117511.0
2,gaming,1070583.0
3,sports,4435425.0
4,how-to,7627111.0
5,business,3455285.0
6,tutorial,8386044.0
7,finance,516184.0
8,crypto,426068.0
9,cubes,15153060.0


### What are the most-liked comments?

#### Query: Select Comment,Likes from comment order by Likes desc

In [13]:
cm_query = text("Select Comment,Likes from comment order by Likes desc")
op = pd.DataFrame(conn.execute(cm_query))
op

,Comment,Likes
0,"Like I said in the video, subscribe if you hav...",891372.0
1,"Huge props to the set designers, everything wa...",469175.0
2,"This is insanity, never done before in the his...",378181.0
3,"Like I said in the video, subscribe if you hav...",268266.0
4,Subscribe and i'll give you a car,260871.0
...,...,...
18404,Awesome Funny Compilation 😻😹😻,0.0
18405,Lovely and funny video,0.0
18406,"6:25 not cute, obviously the dog feels bad.",0.0
18407,Hy dear you're this channel is monetize?,0.0


### What is the ratio of likes/views per video?

#### Query: Select Title,(Likes/Views) as Likes_per_View from videostat

In [14]:
query = text("Select Title,(Likes/Views) as Likes_per_View from videostat")
op = pd.DataFrame(conn.execute(query))
op

,Title,Likes_per_View
0,Apple Pay Is Killing the Physical Wallet After...,0.025123
1,The most EXPENSIVE thing I own.,0.043672
2,My New House Gaming Setup is SICK!,0.040809
3,Petrol Vs Liquid Nitrogen | Freezing Experimen...,0.077543
4,Best Back to School Tech 2022!,0.052011
...,...,...
1876,Should You Learn Machine Learning?,0.026553
1877,Todos podemos aprender Machine learning,0.006910
1878,"Andrew Ng: Deep Learning, Education, and Real-...",0.022985
1879,What is Machine Learning?,NaN


### What is the ratio of likes/views Per each category?

#### Query: Select Keyword,avg(Likes/Views) as Likes_per_Views from videostat group by Keyword

In [15]:
query = text("Select Keyword,avg(Likes/Views) as Likes_per_Views from videostat group by Keyword")
op = pd.DataFrame(conn.execute(query))
op

,Keyword,Likes_per_Views
0,tech,0.033826
1,news,0.011782
2,gaming,0.065196
3,sports,0.012182
4,how-to,0.033896
5,business,0.028853
6,tutorial,0.032555
7,finance,0.018510
8,crypto,0.070752
9,cubes,0.028831


### What is the average sentiment score in each keyword category?

#### Query: Select v.Keyword,avg(c.Sentiment) as Average_Sentiment from videostat as v join comment as c where v.Video_ID = c.Video_ID group by v.Keyword

In [16]:
query = text("Select v.Keyword,avg(c.Sentiment) as Average_Sentiment from videostat as v join comment as c where v.Video_ID = c.Video_ID group by v.Keyword")
op = pd.DataFrame(conn.execute(query))
op

,Keyword,Average_Sentiment
0,tech,1.497817
1,gaming,1.522248
2,news,0.813953
3,minecraft,1.453094
4,asmr,1.742331
5,sports,1.490110
6,how-to,1.554192
7,cubes,1.429412
8,business,1.495614
9,tutorial,1.707843


### How many times do company names (i.e., Apple or Samsung) appear in each keyword category?

#### Query: Select Keyword,count(*) as Total_Count from videostat where Keyword IN('apple','google','xbox','marvel') group by Keyword

In [17]:
query = text("Select Keyword,count(*) as Total_Count from videostat where Keyword IN('apple','google','xbox','marvel') group by Keyword")
op = pd.DataFrame(conn.execute(query))
op

,Keyword,Total_Count
0,apple,42
1,google,45
2,xbox,49
3,marvel,50


## Self defined Queries

### Find out average likes per day for each video

#### Query: Select Title,Likes/datediff(curdate(),str_to_date(Published_At,'%d-%m-%y')) as Average_Likes_per_Day from videostat order by Average_Likes_per_Day desc

In [18]:
query = text("Select Title,Likes/datediff(curdate(),str_to_date(Published_At,'%d-%m-%y')) as Average_Likes_per_Day from videostat order by Average_Likes_per_Day desc")
op = pd.DataFrame(conn.execute(query))
op

,Title,Average_Likes_per_Day
0,"$456,000 Squid Game In Real Life!",26163.363303
1,I Built Willy Wonka&#39;s Chocolate Factory!,12730.798867
2,"I Gave My 100,000,000th Subscriber An Island",12708.828767
3,BLACKPINK - â€˜Pink Venomâ€™ DANCE PRACTICE VIDEO,11034.062500
4,BLACKPINK - â€˜Pink Venomâ€™ DANCE PRACTICE VIDEO,11033.941176
...,...,...
1876,à¸•à¸¥à¸²à¸”à¸–à¸¸à¸‡à¸¡à¸·à¸­à¸¢à¸²à¸‡à¸—à¸£à...,-0.003663
1877,15 Kilo ZayÄ±f GÃ¶steren KÄ±z MakyajÄ± |Arkada...,-0.003663
1878,Vikram Vedha Movie Teaser Review | KRK | #krkr...,-0.003676
1879,What is Machine Learning?,NaN


### Find out the most liked comment for each video and the respective video title

#### Query: SELECT v.Title,v.Video_Id, c.Comment, c.Likes FROM videostat v JOIN (SELECT Video_ID, MAX(Likes) AS max_likes FROM comment GROUP BY Video_ID) t ON v.Video_ID = t.Video_ID JOIN comment c ON t.Video_ID = c.Video_ID AND t.max_likes = c.Likes

In [19]:
query = text("""SELECT v.Title,v.Video_Id, c.Comment, c.Likes
FROM videostat v
JOIN (
    SELECT Video_ID, MAX(Likes) AS max_likes
    FROM comment
    GROUP BY Video_ID
) t ON v.Video_ID = t.Video_ID
JOIN comment c ON t.Video_ID = c.Video_ID AND t.max_likes = c.Likes
""")
op = pd.DataFrame(conn.execute(query))
op

,Title,Video_Id,Comment,Likes
0,Apple Pay Is Killing the Physical Wallet After...,wAZZ-UWGVHI,I will forever acknowledge this channel with t...,161.0
1,The most EXPENSIVE thing I own.,b3x28s61q3c,I love all the people LTT has added for labs. ...,2808.0
2,My New House Gaming Setup is SICK!,4mgePWWCAmA,Dan the man. Saving the day. Riley needs to gi...,3371.0
3,My New House Gaming Setup is SICK!,4mgePWWCAmA,Dan the man. Saving the day. Riley needs to gi...,3371.0
4,Petrol Vs Liquid Nitrogen | Freezing Experimen...,kXiYSI7H2b0,Liquid nitrogen ഇതുവരെ തീർന്നില്ലേ മച്ചാനെ.......,3660.0
...,...,...,...,...
1943,Should You Learn Machine Learning?,AO6urf07KjE,Pros : It is fun\r\n\r\nCons : You have to wor...,115.0
1944,Todos podemos aprender Machine learning,7ClLKBUvmRk,"Me resultó tan inspirador conocerte, Haydé! Me...",127.0
1945,"Andrew Ng: Deep Learning, Education, and Real-...",0jspaMLxBig,"Whenever someone uses the word ""It turns out"",...",359.0
1946,What is Machine Learning?,f_uwKZIAeM0,Machine learning is an incredibly powerful num...,38.0
